In [71]:
import pandas as pd
import datetime as dt

In [72]:
start_date = dt.date(1992,1,1)
end_date = dt.date(2019,1,1)
# Use the start and end date to create a range of dates
dates_range = pd.date_range(start_date, end_date, freq='Y')
# Strip off the year and save a list of %m-%d strings
dates_range = dates_range.strftime("%Y")
dates_range

array(['1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999',
       '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017', '2018'], dtype='<U4')

In [73]:
data_dict = {}
team_id = "COL"
for year in dates_range:
    url = f"https://www.baseball-reference.com/draft/?team_ID={team_id}&year_ID={year}&draft_type=junreg&query_type=franch_year&from_type_jc=0&from_type_hs=0&from_type_4y=0&from_type_unk=0"
    tables = pd.read_html(url)
    data_dict[year] = pd.DataFrame(tables[0])

In [74]:
for year in dates_range:
    data_dict[year].fillna(0, inplace=True)
    data_dict[year] = data_dict[year].drop(columns=['DT'])

In [75]:
frames = [data_dict['1992'],
          data_dict['1993'],data_dict['1994'],data_dict['1995'],data_dict['1996'],data_dict['1997'],data_dict['1998'],data_dict['1999'],
          data_dict['2000'],data_dict['2001'],data_dict['2002'],data_dict['2003'],data_dict['2004'],data_dict['2005'],data_dict['2006'],
          data_dict['2007'],data_dict['2008'],data_dict['2009'],data_dict['2010'],data_dict['2011'],data_dict['2012'],data_dict['2013'],
          data_dict['2014'],data_dict['2015'],data_dict['2016'],data_dict['2017'],data_dict['2018']]
full_df = pd.concat(frames)
full_df.head()

,Year,Rnd,OvPck,RdPck,Tm,Signed,Name,Pos,WAR,G,...,BA,OPS,G.1,W,L,ERA,WHIP,SV,Type,Drafted Out of
0,1992,1,27,27,Rockies,Y,John Burke (minors),RHP,-0.5,28.0,...,0.190,0.381,28.0,4.0,6.0,6.75,1.83,0.0,4Yr,"University of Florida (Gainesville, FL)"
1,1992,2,65,27,Rockies,Y,Mark Thompson (minors),RHP,0.0,93.0,...,0.154,0.383,94.0,18.0,24.0,5.74,1.68,0.0,4Yr,"University of Kentucky (Lexington, KY)"
2,1992,3,95,27,Rockies,Y,Roger Bailey (minors),RHP,5.2,92.0,...,0.206,0.520,92.0,18.0,19.0,4.90,1.55,1.0,4Yr,"Florida State University (Tallahassee, FL)"
3,1992,4,123,27,Rockies,Y,Lloyd Peever (minors),RHP,0.0,0.0,...,0.000,0.000,0.0,0.0,0.0,0.00,0.00,0.0,4Yr,"Louisiana State University (Baton Rouge, LA)"
4,1992,5,151,27,Rockies,Y,Ryan Freeburg (minors),3B,0.0,0.0,...,0.000,0.000,0.0,0.0,0.0,0.00,0.00,0.0,4Yr,"Grand Canyon University (Phoenix, AZ)"


In [76]:
# frames = [data_dict['1965'], data_dict['1966'],data_dict['1967'],data_dict['1968'],data_dict['1969'],data_dict['1970'],data_dict['1971'],
#           data_dict['1972'],data_dict['1973'],data_dict['1974'],data_dict['1975'],data_dict['1976'],data_dict['1977'],data_dict['1978'],
#           data_dict['1979'],data_dict['1980'],data_dict['1981'],data_dict['1982'],data_dict['1983'],data_dict['1984'],data_dict['1985'],
#           data_dict['1986'],data_dict['1987'],data_dict['1988'],data_dict['1989'],data_dict['1990'],data_dict['1991'],data_dict['1992'],
#           data_dict['1993'],data_dict['1994'],data_dict['1995'],data_dict['1996'],data_dict['1997'],data_dict['1998'],data_dict['1999'],
#           data_dict['2000'],data_dict['2001'],data_dict['2002'],data_dict['2003'],data_dict['2004'],data_dict['2005'],data_dict['2006'],
#           data_dict['2007'],data_dict['2008'],data_dict['2009'],data_dict['2010'],data_dict['2011'],data_dict['2012'],data_dict['2013'],
#           data_dict['2014'],data_dict['2015'],data_dict['2016'],data_dict['2017'],data_dict['2018']]
# full_df = pd.concat(frames)
# full_df.head()

In [77]:
school = pd.DataFrame({'location': full_df['Drafted Out of'], 'Name': full_df['Name']})

school['school'] = school.location.str.split('(').str.get(0)
school['city'] = school.location.str.split('(').str.get(1)
school['town'] = school.city.str.split(',').str.get(0)
school['states'] = school.city.str.split(',').str.get(1)
del school['location']
del school['city']
school['state'] = school.states.str.split(')').str.get(0)
school['ST'] = school.state.str.split(' ').str.get(1)
del school["state"]
del school["states"]
school.head()

,Name,school,town,ST
0,John Burke (minors),University of Florida,Gainesville,FL
1,Mark Thompson (minors),University of Kentucky,Lexington,KY
2,Roger Bailey (minors),Florida State University,Tallahassee,FL
3,Lloyd Peever (minors),Louisiana State University,Baton Rouge,LA
4,Ryan Freeburg (minors),Grand Canyon University,Phoenix,AZ


In [78]:
df = full_df.merge(school, left_on='Name', right_on='Name', how='left')
del df['Drafted Out of']
df.head()

,Year,Rnd,OvPck,RdPck,Tm,Signed,Name,Pos,WAR,G,...,G.1,W,L,ERA,WHIP,SV,Type,school,town,ST
0,1992,1,27,27,Rockies,Y,John Burke (minors),RHP,-0.5,28.0,...,28.0,4.0,6.0,6.75,1.83,0.0,4Yr,University of Florida,Gainesville,FL
1,1992,2,65,27,Rockies,Y,Mark Thompson (minors),RHP,0.0,93.0,...,94.0,18.0,24.0,5.74,1.68,0.0,4Yr,University of Kentucky,Lexington,KY
2,1992,3,95,27,Rockies,Y,Roger Bailey (minors),RHP,5.2,92.0,...,92.0,18.0,19.0,4.90,1.55,1.0,4Yr,Florida State University,Tallahassee,FL
3,1992,4,123,27,Rockies,Y,Lloyd Peever (minors),RHP,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.0,4Yr,Louisiana State University,Baton Rouge,LA
4,1992,5,151,27,Rockies,Y,Ryan Freeburg (minors),3B,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.0,4Yr,Grand Canyon University,Phoenix,AZ


In [79]:
df.reset_index(drop=True)

,Year,Rnd,OvPck,RdPck,Tm,Signed,Name,Pos,WAR,G,...,G.1,W,L,ERA,WHIP,SV,Type,school,town,ST
0,1992,1,27,27,Rockies,Y,John Burke (minors),RHP,-0.5,28.0,...,28.0,4.0,6.0,6.75,1.83,0.0,4Yr,University of Florida,Gainesville,FL
1,1992,2,65,27,Rockies,Y,Mark Thompson (minors),RHP,0.0,93.0,...,94.0,18.0,24.0,5.74,1.68,0.0,4Yr,University of Kentucky,Lexington,KY
2,1992,3,95,27,Rockies,Y,Roger Bailey (minors),RHP,5.2,92.0,...,92.0,18.0,19.0,4.90,1.55,1.0,4Yr,Florida State University,Tallahassee,FL
3,1992,4,123,27,Rockies,Y,Lloyd Peever (minors),RHP,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.0,4Yr,Louisiana State University,Baton Rouge,LA
4,1992,5,151,27,Rockies,Y,Ryan Freeburg (minors),3B,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.0,4Yr,Grand Canyon University,Phoenix,AZ
5,1992,6,179,27,Rockies,Y,Will Scalzitti (minors),C,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.0,JC,Miami-Dade College,Miami,FL
6,1992,7,207,27,Rockies,Y,Jason Bates (minors),SS,-2.6,319.0,...,0.0,0.0,0.0,0.00,0.00,0.0,4Yr,University of Arizona,Tucson,AZ
7,1992,8,235,27,Rockies,Y,Chris Henderson (minors),RHP,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.0,JC,Eastern Florida State College,Cocoa,FL
8,1992,9,263,27,Rockies,Y,Mike Eiffert (minors),LHP,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.0,4Yr,University of Texas Pan American,Edinburg,TX
9,1992,10,291,27,Rockies,Y,Garvin Alston (minors),RHP,-0.1,6.0,...,6.0,1.0,0.0,9.00,2.00,0.0,4Yr,Florida International University,Miami,FL


In [80]:
df.to_csv(f"Resources/complete_{team_id}_drafts.csv")